In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import plotly.io as pio

from scipy import stats
from scipy.constants import euler_gamma
from scipy.special import erf

pio.templates.default = "plotly_white"
pio.renderers.default = os.environ.get("PLOTLY_RENDERER", "notebook")
np.set_printoptions(precision=4, suppress=True)

rng = np.random.default_rng(42)


ImportError: cannot import name 'euler_gamma' from 'scipy.constants' (/home/tempa/miniconda3/lib/python3.12/site-packages/scipy/constants/__init__.py)

In [2]:
def rayleigh_pdf(x, sigma):
    """PDF of Rayleigh(sigma), support x >= 0."""
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    out = np.zeros_like(x, dtype=float)
    mask = x >= 0
    xm = x[mask]
    out[mask] = (xm / sigma**2) * np.exp(-(xm**2) / (2 * sigma**2))
    return out


def rayleigh_cdf(x, sigma):
    """CDF of Rayleigh(sigma), stable near 0 via expm1."""
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    out = np.zeros_like(x, dtype=float)
    mask = x >= 0
    xm = x[mask]
    out[mask] = -np.expm1(-(xm**2) / (2 * sigma**2))
    return out


def rayleigh_logpdf(x, sigma):
    """Log-PDF of Rayleigh(sigma). Returns -inf for x <= 0."""
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    out = np.full_like(x, -np.inf, dtype=float)
    mask = x > 0
    xm = x[mask]
    out[mask] = np.log(xm) - 2 * np.log(sigma) - (xm**2) / (2 * sigma**2)
    return out


def rayleigh_ppf(u, sigma):
    """Inverse CDF (percent point function), for u in [0, 1)."""
    u = np.asarray(u, dtype=float)
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    if np.any((u < 0) | (u >= 1)):
        raise ValueError("u must be in [0, 1)")

    return sigma * np.sqrt(-2.0 * np.log1p(-u))


In [3]:
def rayleigh_mean(sigma):
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    return sigma * np.sqrt(np.pi / 2)


def rayleigh_var(sigma):
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    return ((4 - np.pi) / 2) * sigma**2


def rayleigh_skewness():
    return (2 * np.sqrt(np.pi) * (np.pi - 3)) / (4 - np.pi) ** (3 / 2)


def rayleigh_excess_kurtosis():
    return (-6 * np.pi**2 + 24 * np.pi - 16) / (4 - np.pi) ** 2


def rayleigh_entropy(sigma):
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    return 1 + np.log(sigma / np.sqrt(2)) + 0.5 * euler_gamma


def rayleigh_mgf(t, sigma):
    """MGF M(t) = E[exp(tX)] for X ~ Rayleigh(sigma). Works for real or complex t."""
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")

    t = np.asarray(t)
    z = sigma * t / np.sqrt(2)
    return 1 + sigma * t * np.sqrt(np.pi / 2) * np.exp(0.5 * (sigma * t) ** 2) * (1 + erf(z))


# Quick numeric check against Monte Carlo
sigma = 1.7
n = 200_000
samples = rayleigh_ppf(rng.random(n), sigma=sigma)

print("Mean (theory, MC):", rayleigh_mean(sigma), samples.mean())
print("Var  (theory, MC):", rayleigh_var(sigma), samples.var())
print("Skewness (theory):", rayleigh_skewness())
print("Excess kurtosis (theory):", rayleigh_excess_kurtosis())
print("Entropy (theory):", rayleigh_entropy(sigma))

# MGF check at a couple of points
for t in [0.2, 0.8]:
    mc = np.mean(np.exp(t * samples))
    th = rayleigh_mgf(t, sigma)
    print(f"MGF at t={t}: theory={th:.6f}, MC={mc:.6f}")


NameError: name 'rng' is not defined

In [4]:
sigmas = [0.5, 1.0, 2.0]

x = np.linspace(0, 10, 600)

# PDF
fig = go.Figure()
for s in sigmas:
    fig.add_trace(go.Scatter(x=x, y=rayleigh_pdf(x, s), mode="lines", name=f"σ={s:g}"))

fig.update_layout(
    title="Rayleigh PDF: effect of the scale σ",
    xaxis_title="x",
    yaxis_title="pdf",
)
fig.show()

# CDF
fig = go.Figure()
for s in sigmas:
    fig.add_trace(go.Scatter(x=x, y=rayleigh_cdf(x, s), mode="lines", name=f"σ={s:g}"))

fig.update_layout(
    title="Rayleigh CDF: effect of the scale σ",
    xaxis_title="x",
    yaxis_title="cdf",
)
fig.show()


In [5]:
def rayleigh_loglik(x, sigma):
    x = np.asarray(x, dtype=float)
    sigma = float(sigma)
    if sigma <= 0 or np.any(x < 0):
        return -np.inf
    return np.sum(rayleigh_logpdf(x, sigma))


def rayleigh_mle_sigma(x):
    x = np.asarray(x, dtype=float)
    if np.any(x < 0):
        raise ValueError("Rayleigh data must be nonnegative")
    return np.sqrt(np.mean(x**2) / 2)


# MLE demo
sigma_true = 1.8
x = rayleigh_ppf(rng.random(5000), sigma=sigma_true)

sigma_hat = rayleigh_mle_sigma(x)

# Compare to SciPy's fit (fix loc=0 to match our parameterization)
loc_hat, scale_hat = stats.rayleigh.fit(x, floc=0)

print("sigma_true:", sigma_true)
print("sigma_hat (closed form):", sigma_hat)
print("sigma_hat (scipy fit):   ", scale_hat)
print("loc_hat (scipy fit):     ", loc_hat)


NameError: name 'rng' is not defined

In [6]:
def rayleigh_rvs_numpy(sigma, size, rng=None):
    """NumPy-only Rayleigh sampler via inverse CDF."""
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    if rng is None:
        rng = np.random.default_rng()

    u = rng.random(size)
    return rayleigh_ppf(u, sigma=sigma)


def rayleigh_rvs_via_normals(sigma, size, rng=None):
    """Sampler using the 2D Gaussian magnitude definition."""
    sigma = float(sigma)
    if sigma <= 0:
        raise ValueError("sigma must be > 0")
    if rng is None:
        rng = np.random.default_rng()

    if isinstance(size, tuple):
        shape = (2, *size)
    else:
        shape = (2, size)

    xy = rng.normal(loc=0.0, scale=sigma, size=shape)
    return np.sqrt(xy[0] ** 2 + xy[1] ** 2)


# Quick sampling sanity check
sigma = 1.3
s1 = rayleigh_rvs_numpy(sigma, size=200_000, rng=rng)
print("Mean (theory, sampler):", rayleigh_mean(sigma), s1.mean())


NameError: name 'rng' is not defined

In [7]:
def ecdf(samples):
    x = np.sort(np.asarray(samples))
    y = np.arange(1, x.size + 1) / x.size
    return x, y


sigma_viz = 1.6
n_viz = 120_000
samples = rayleigh_rvs_numpy(sigma_viz, size=n_viz, rng=rng)

# Histogram + PDF overlay
x_grid = np.linspace(0, np.quantile(samples, 0.999), 600)

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=samples,
        nbinsx=80,
        histnorm="probability density",
        name="Monte Carlo",
        opacity=0.6,
    )
)
fig.add_trace(
    go.Scatter(
        x=x_grid,
        y=rayleigh_pdf(x_grid, sigma_viz),
        mode="lines",
        name="theoretical pdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title=f"Rayleigh PDF vs Monte Carlo (σ={sigma_viz:g}, n={n_viz:,})",
    xaxis_title="x",
    yaxis_title="density",
)
fig.show()

# ECDF vs CDF
x_ecdf, y_ecdf = ecdf(samples)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_ecdf, y=y_ecdf, mode="lines", name="ECDF"))
fig.add_trace(
    go.Scatter(
        x=x_grid,
        y=rayleigh_cdf(x_grid, sigma_viz),
        mode="lines",
        name="theoretical cdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title="CDF check: ECDF vs theoretical",
    xaxis_title="x",
    yaxis_title="cdf",
)
fig.show()

# Squaring property: X^2 ~ Exp(rate = 1/(2σ^2))
rate = 1 / (2 * sigma_viz**2)
y = samples**2
x_y = np.linspace(0, np.quantile(y, 0.999), 600)

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=y,
        nbinsx=80,
        histnorm="probability density",
        name="Y = X² (MC)",
        opacity=0.6,
    )
)
fig.add_trace(
    go.Scatter(
        x=x_y,
        y=rate * np.exp(-rate * x_y),
        mode="lines",
        name=f"Exp(rate={rate:.3g}) pdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title="Squaring property: Y = X² is exponential",
    xaxis_title="y",
    yaxis_title="density",
)
fig.show()


NameError: name 'rng' is not defined

In [8]:
sigma = 1.4

dist = stats.rayleigh(loc=0, scale=sigma)

x = np.linspace(0, 8, 400)
pdf_scipy = dist.pdf(x)
cdf_scipy = dist.cdf(x)

# Compare SciPy vs our implementations
print("max |pdf diff|:", np.max(np.abs(pdf_scipy - rayleigh_pdf(x, sigma))))
print("max |cdf diff|:", np.max(np.abs(cdf_scipy - rayleigh_cdf(x, sigma))))

# Sampling
rvs = dist.rvs(size=5, random_state=rng)
print("rvs:", rvs)

# Fitting (MLE). Fix loc=0 for canonical Rayleigh.
true_sigma = 2.0
data = stats.rayleigh(scale=true_sigma).rvs(size=4000, random_state=rng)
loc_hat, scale_hat = stats.rayleigh.fit(data, floc=0)
print("true sigma:", true_sigma)
print("fit scale (sigma_hat):", scale_hat)


max |pdf diff|: 1.6653345369377348e-16
max |cdf diff|: 2.220446049250313e-16


NameError: name 'rng' is not defined

In [9]:
# A) Exact test + confidence interval for sigma

sigma_true = 1.5
n = 400
x = stats.rayleigh(scale=sigma_true).rvs(size=n, random_state=rng)

sigma0 = 1.3  # null hypothesis

test_stat = np.sum(x**2) / sigma0**2
p_lower = stats.chi2.cdf(test_stat, df=2 * n)
p_upper = 1 - p_lower
p_two_sided = 2 * min(p_lower, p_upper)

print(f"H0: sigma = {sigma0:g}")
print("test statistic:", test_stat)
print("two-sided p-value:", p_two_sided)

# 95% CI for sigma via chi-square quantiles
alpha = 0.05
s = np.sum(x**2)
q_lo = stats.chi2.ppf(alpha / 2, df=2 * n)
q_hi = stats.chi2.ppf(1 - alpha / 2, df=2 * n)

sigma_ci_lo = np.sqrt(s / q_hi)
sigma_ci_hi = np.sqrt(s / q_lo)

print(f"95% CI for sigma: [{sigma_ci_lo:.4f}, {sigma_ci_hi:.4f}]")
print("true sigma:", sigma_true)


NameError: name 'rng' is not defined

In [10]:
# B) Bayesian update using Y = X^2 ~ Exp(rate = 1/(2 sigma^2))

sigma_true = 1.7
n = 200
x = stats.rayleigh(scale=sigma_true).rvs(size=n, random_state=rng)

y = x**2

# Prior on lambda = 1/(2 sigma^2): Gamma(shape=a0, rate=b0)
a0, b0 = 2.0, 1.0

# Exponential likelihood: p(y | lambda) = lambda * exp(-lambda y)
a_post = a0 + n
b_post = b0 + y.sum()

# Posterior samples of lambda, then transform to sigma
m = 50_000
lambda_samps = rng.gamma(shape=a_post, scale=1 / b_post, size=m)  # numpy uses scale=1/rate
sigma_samps = 1 / np.sqrt(2 * lambda_samps)

ci = np.quantile(sigma_samps, [0.025, 0.5, 0.975])

print("posterior median sigma:", ci[1])
print("95% credible interval:", (ci[0], ci[2]))
print("true sigma:", sigma_true)


NameError: name 'rng' is not defined

In [11]:
# C) Generative modeling: I/Q Gaussian -> Rayleigh amplitude

sigma = 1.2
n = 80_000

i = rng.normal(loc=0.0, scale=sigma, size=n)
q = rng.normal(loc=0.0, scale=sigma, size=n)
amp = np.sqrt(i**2 + q**2)

x_grid = np.linspace(0, np.quantile(amp, 0.999), 600)

fig = go.Figure()
fig.add_trace(
    go.Histogram(
        x=amp,
        nbinsx=80,
        histnorm="probability density",
        name="|I + jQ| (MC)",
        opacity=0.6,
    )
)
fig.add_trace(
    go.Scatter(
        x=x_grid,
        y=rayleigh_pdf(x_grid, sigma),
        mode="lines",
        name="Rayleigh pdf",
        line=dict(width=3),
    )
)
fig.update_layout(
    title="Amplitude of complex Gaussian noise is Rayleigh",
    xaxis_title="amplitude",
    yaxis_title="density",
)
fig.show()


NameError: name 'rng' is not defined